In [1]:
# load pd df from csv of jlc_parts.csv

import pandas as pd
import numpy as np

# load csv
df = pd.read_csv('jlc_parts.csv', sep=',', header=0, index_col=0)

In [2]:
# list unique column names
print(df.columns)
# df['Category'].unique()

# remove rows with Resistor Networks & Arrays in Category
# df = df[df.Category != 'Chip Resistor - Surface Mount ( UniOhm)']


Index(['LCSC Part #', 'Description', 'value', 'tolerance', 'type', 'voltage',
       'min_value', 'max_value', 'Package', 'Stock', 'Type', 'MFR.Part #',
       'Datasheet', 'Price (USD)', 'basic-extended', 'unit', 'Comment',
       'Category', 'code', 'value_in_ohms', 'dielectric', 'value_in_farads'],
      dtype='object')


In [3]:
# load into firestore table
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account.
cred = credentials.Certificate('atopile-880ca67acfe2.json')

app = firebase_admin.initialize_app(cred)

db = firestore.client()

# add data to firestore with these fields
# Index(['LCSC Part #', 'Description', 'value', 'tolerance', 'type', 'voltage',
#        'min_value', 'max_value', 'Package', 'Stock', 'Type', 'MFR.Part #',
#        'Datasheet', 'Price (USD)', 'basic-extended', 'unit', 'Comment',
#        'Category', 'code', 'value_in_ohms', 'dielectric', 'value_in_farads'],
#       dtype='object')

# make a collection for resistors
# db.collection("resistors")
# set the document id to the LCSC Part #
for index, row in df.iterrows():
    doc_ref = db.collection(u'components').document(row['LCSC Part #'])
    doc_ref.set({
        u'LCSC Part #': row['LCSC Part #'],
        u'Description': row['Description'],
        u'value': row['value'],
        u'tolerance': row['tolerance'],
        u'type': row['type'],
        u'voltage': row['voltage'],
        u'min_value': row['min_value'],
        u'max_value': row['max_value'],
        u'Package': row['Package'],
        u'Stock': row['Stock'],
        u'Type': row['Type'],
        u'MFR.Part #': row['MFR.Part #'],
        u'Datasheet': row['Datasheet'],
        u'Price (USD)': row['Price (USD)'],
        u'basic-extended': row['basic-extended'],
        u'unit': row['unit'],
        u'Comment': row['Comment'],
        u'Category': row['Category'],
        u'code': row['code'],
        u'value_in_ohms': row['value_in_ohms'],
        u'dielectric': row['dielectric'],
        u'value_in_farads': row['value_in_farads']
    })

# make a table for capacitors

In [6]:
import pandas as pd

df = pd.read_html('https://jlcpcb.com/partdetail/C10050')
df

ValueError: No tables found matching pattern '.+'

In [61]:
from bs4 import BeautifulSoup

import requests

url = 'https://jlcpcb.com/partdetail/C2040'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# # find lcsc part number

tables = soup.find_all('table')
len(tables)
tables[1]
# Parse the HTML content
# soup = BeautifulSoup(response.text, 'html.parser')

# # Dictionary to store the data
# part_details = {}

# # Find all the 'dl' elements
# for dl in soup.find_all('dl', class_='pt-[23px] pl-30 pr-30 pb-30"'):
#     # Extract the 'dt' (title) and 'dd' (description) elements
#     dt = dl.find('dt').get_text(strip=True)
#     dd = dl.find('dd').get_text(strip=True)

#     # Add the extracted information to the dictionary
#     part_details[dt] = dd

# # Print the dictionary
# for key, value in part_details.items():
#     print(f"{key}: {value}")<div class="el-table__body-wrapper is-scrolling-none"><table border="0" cellpadding="0" cellspacing="0" class="el-table__body" style="width:;"><colgroup></colgroup><tbody><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><!-- --></tbody></table><!-- --><!-- --></div>


<table border="0" cellpadding="0" cellspacing="0" class="el-table__body" style="width:;"><colgroup></colgroup><tbody><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-table__row"></tr><tr class="el-t

In [50]:
from bs4 import BeautifulSoup

import requests

url = 'https://jlcpcb.com/partdetail/C100082'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Extracting stock information
stock_info = soup.find("div", class_="text-16 font-bold")
stock = int(stock_info.text.strip().split(': ')[1]) if stock_info else None

# Extracting price information
price_divs = soup.find_all("div", class_="flex items-center justify-between mt-14")
price = []

for div in price_divs:
    # Extract quantity and price
    qty = div.find("span", class_="w-120 inline-block").text.strip()
    pr = div.find("span", class_="w-120 text-left").text.strip().replace('$', '')
    
    # Add to the list as a tuple
    price.append((qty, float(pr)))

# Printing the results
print("Stock:", stock)
print("Price List:", price)

Stock: 7027
Price List: [('1+', 0.015), ('500+', 0.0123), ('1500+', 0.0108), ('4000+', 0.0099), ('24000+', 0.0091), ('48000+', 0.0087)]


In [40]:
def get_lcsc_data(lcsc_id):
    # Construct the URL
    url = 'https://jlcpcb.com/partdetail/' + lcsc_id
    # get the HTML content
    response = requests.get(url)
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    # Dictionary to store the data
    part_details = {}
    # Find all the 'dl' elements
    for dl in soup.find_all('dl', class_='flex dl-list'):
        # Extract the 'dt' (title) and 'dd' (description) elements
        dt = dl.find('dt').get_text(strip=True)
        dd = dl.find('dd').get_text(strip=True)

        # Add the extracted information to the dictionary
        part_details[dt] = dd

    # check if the dictionary is empty
    # if all the dictionaries are empty, then the part ID is invalid
    # example empty row {'Manufacturer': '', 'MFR.Part #': '', 'JLCPCB Part #': '', 'Package': '', 'Description': '', 'Datasheet': 'Download', 'Source': 'JLCPCB', 'Assembly Type': 'A PCB assembly fixture is needed to protect and support this part during the assembly process.', 'CAD Model': 'PCB Footprint or Symbol'}
    # Check if 'JLCPCB Part #' is empty
    if not part_details.get('JLCPCB Part #'):
        part_details = None
    return part_details

In [43]:
get_lcsc_data('C1529')

for x in range(1100,1160):
    data = get_lcsc_data('C'+str(x))
    if data:
        print(data)

{'Manufacturer': 'FH (Guangdong Fenghua Advanced Tech)', 'MFR.Part #': 'RS-02K820JT', 'JLCPCB Part #': 'C1102', 'Package': 'R0402', 'Description': '0402  Chip Resistor - Surface Mount ROHS', 'Datasheet': 'Download', 'Source': 'JLCPCB', 'Assembly Type': 'SMT AssemblyA PCB assembly fixture is needed to protect and support this part during the assembly process.', 'CAD Model': 'PCB Footprint or Symbol'}
{'Manufacturer': 'FH (Guangdong Fenghua Advanced Tech)', 'MFR.Part #': 'RC-02W221JT', 'JLCPCB Part #': 'C1111', 'Package': '0402', 'Description': '62.5mW ±5% ±200ppm/℃ 220Ω 0402  Chip Resistor - Surface Mount ROHS', 'Datasheet': 'Download', 'Source': 'JLCPCB', 'Assembly Type': 'SMT AssemblyA PCB assembly fixture is needed to protect and support this part during the assembly process.', 'CAD Model': 'PCB Footprint or Symbol'}
{'Manufacturer': 'FH(Guangdong Fenghua Advanced Tech)', 'MFR.Part #': 'RS-02W271JT', 'JLCPCB Part #': 'C1113', 'Package': 'R0402', 'Description': '±5% ±200ppm/℃ -55℃~+15